In [11]:
import pandas as pd
from copy import deepcopy
from pathlib import Path
from scipy.stats import ttest_ind, fisher_exact
from Bio.Seq import Seq
from Bio import SeqIO
from statsmodels.stats.multitest import multipletests
import numpy as np
from itertools import chain


In [12]:
# we're actually going to drop the clearance time in file in favor of a centralized one
def clean_var_data(var_data):
    return var_data[['POS'] + [x for x in var_data.columns if x.isnumeric()]]

# we need to summarize the variants data and do the one-way anova before trying anything else
def process_var_data(variants, labels, ttest=True):
    results = []
    positions = [ x for x in variants.columns if x.isnumeric()]
    for p in positions:
        var_counts = variants[p].value_counts()
        wt_names = [ x for x in variants.loc[variants[p] == var_counts.index[0], 'POS'].values if x in labels.index] # the pos column is actually the name column
        if len(var_counts) > 1:
            for idx in var_counts.index[1:]: # idx is the variant
                var_names = [ x for x in variants.loc[variants[p] == idx, 'POS'].values if x in labels.index]
                assert abs(len(var_names) - var_counts[idx]) <= 1 # can be off by one cuz we don't have some labels
                if ttest:
                    _, p_val = ttest_ind(
                        [float(x) for x in labels.loc[wt_names] if not pd.isna(x)],
                        [float(x) for x in labels.loc[var_names] if not pd.isna(x)],
                        equal_var=False, nan_policy='omit', 
                        alternative='two-sided'
                    )
                else:
                    #binary variables need chisquared
                    wt_labels = [float(x) for x in labels.loc[wt_names] if not pd.isna(x)]
                    var_labels = [float(x) for x in labels.loc[var_names] if not pd.isna(x)]
                    contingency_table = np.array([
                        [sum(wt_labels), (len(wt_labels) - sum(wt_labels))],
                        [sum(var_labels), (len(var_labels) - sum(var_labels))],
                    ])
                    if np.min(contingency_table) > 0:
                        p_val = fisher_exact(contingency_table)[1]
                    else:
                        p_val = 1 # if we don't have enough data to get a pval it's a pass
                results.append({
                    'pos': p,
                    'wt': var_counts.index[0],
                    'var': idx,
                    'count': len(var_names),
                    'p_val': p_val,
                })
    return pd.DataFrame(results)


# gene_data is a slice of the dataframe
def check_synonymous(variants, gene_data, sequence, prot_sequence):
    # the reverse genes are on the complement trans and backwords, so we'll need
    # to change the nucleotide as well
    complement_map = {
        'A': 'T',
        'T': 'A',
        'C': 'G',
        'G': 'C',
    }

    if len(gene_data['direction'].unique()) != 1:
        raise ValueError('Multiple directions')
    
    is_forward = gene_data['direction'].unique()[0] == 'forward'
    # construct the nucleotide mapping
    cds_positions = list(chain.from_iterable([
        range(start, end + 1)
        for start, end in 
        zip(gene_data.cds_start.values, gene_data.cds_end.values)
    ]))

    if 'reverse' in gene_data.direction.values:
        cds_positions.reverse()
    
    # here we have genome position: 
    pos_map = {pos: idx for idx, pos in enumerate(cds_positions)}

    syn_results = []
    # now we can apply the variants
    for r in variants.to_dict('records'):

        # we don't consider variants not in the gene range for this function
        position = int(r['pos'])
        aa_pos = None
        old_aa = None
        new_aa = None
        if position not in pos_map:
            syn = None
        elif len(r['var']) != 1 or len(r['wt']) != 1 or r['var'] == '.' or r['wt'] == '.':
            syn = False  # if the variant is not single nuc, it's always non-syn
        else:
            rp = pos_map[position] # position to replace
            wt = r['wt'] if is_forward else complement_map[r['wt']]
            var = r['var'] if is_forward else complement_map[r['var']]
            wt_seq = sequence[:rp] + wt + sequence[rp + 1:]
            new_seq = sequence[:rp] + var + sequence[rp + 1:] # replace one character
            wt_prot = str(Seq(wt_seq).translate())[:-1]
            new_prot = str(Seq(new_seq).translate())[:-1] # the last char is the stop codon
            try:
                assert len(new_prot) == len(wt_prot) == len(prot_sequence)
            except AssertionError:
                print(r)
                raise AssertionError
            syn = (new_prot == wt_prot)
            aa_pos = int(rp / 3) + 1
            old_aa = wt_prot[aa_pos - 1]
            new_aa = new_prot[aa_pos - 1]
        
        r['is_synom'] = syn
        r['aa_pos'] = aa_pos
        r['old_aa'] = old_aa
        r['new_aa'] = new_aa
        syn_results.append(r)
    
    return pd.DataFrame(syn_results)




In [13]:
# load data
data_path = Path('/mnt/d/data/popnet_paper/variant_data')
plasmo_gene_data = pd.read_csv(data_path / 'plasmo_genes.csv')
neis_gene_data = pd.read_csv(data_path / 'neis_genes.csv')

plasmo_genes = plasmo_gene_data['name'].unique()
neis_genes = neis_gene_data['name'].unique()

In [14]:
# process plasmo genes
plasmo_labels = pd.read_csv(data_path / 'plasmo_meta.tsv', sep='\t', index_col = 0).rename({'Parasites clearance time': 'label'}, axis=1)
results = []
for gene in plasmo_genes:
    var_data = clean_var_data(pd.read_csv(data_path / f"{gene}.csv"))
    with open(data_path / f'{gene}_cds.fasta') as f:
        sequence = [str(x.seq) for x in SeqIO.parse(f, 'fasta')][0] # there is always only one record per file
    with open(data_path / f'{gene}_prot.fasta') as f:
        prot_sequence = [str(x.seq) for x in SeqIO.parse(f, 'fasta')][0]
    var_summary = process_var_data(var_data, plasmo_labels['label'])
    gene_data_slice = plasmo_gene_data.loc[plasmo_gene_data['name'] == gene]
    var_w_syn = check_synonymous(
        var_summary, gene_data_slice, sequence, prot_sequence,
    )
    var_w_syn['chr'] = gene_data_slice['region'].apply(lambda x: x.split('_')[1]).unique()[0]
    var_w_syn['gene'] = gene
    var_w_syn['species'] = 'plasmodium'
    results.append(var_w_syn)


In [15]:
# run analysis for neisseria
labels = pd.read_csv(data_path / 'neis_meta.tsv', sep='\t').set_index('Sample')
gene_data = neis_gene_data
for gene in neis_genes:
    try:
        var_data = clean_var_data(pd.read_csv(data_path / f"{gene}.csv"))
    except FileNotFoundError:
        continue

    var_summary = process_var_data(var_data, labels['label'], ttest=False)
    gene_data_slice = gene_data.loc[gene_data['name'] == gene]
    with open(data_path / f'{gene}_cds.fasta') as f:
        sequence = [str(x.seq) for x in SeqIO.parse(f, 'fasta')][0] # there is always only one record per file
    with open(data_path / f'{gene}_prot.fasta') as f:
        prot_sequence = [str(x.seq) for x in SeqIO.parse(f, 'fasta')][0]

    var_w_syn = check_synonymous(
        var_summary, gene_data_slice, sequence, prot_sequence,
    )

    var_w_syn['chr'] = 1
    var_w_syn['gene'] = gene
    var_w_syn['species'] = 'neisseria'
    results.append(var_w_syn)


In [16]:
results_df = pd.concat(results)
results_df['p_adj'] = multipletests(results_df['p_val'].values)[1]
results_df['is_sig'] = results_df['p_adj'] < 0.01

results_df = results_df[[
    'species', 'gene', 'chr', 'pos', 'wt', 'var', 'p_adj', 'is_sig', 'count', 'aa_pos', 'old_aa', 'new_aa', 'is_synom',
]]

/home/javi/workspace/NeuralNet/venv/lib/python3.7/site-packages/statsmodels/stats/multitest.py:160: RuntimeWarning: invalid value encountered in greater
  notreject = pvals > alphacSidak_all
/home/javi/workspace/NeuralNet/venv/lib/python3.7/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/home/javi/workspace/NeuralNet/venv/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [17]:
# construct the per-segment table
gene_seg_map = {
    'med14': 'PfS1',
    'pf': 'PfS2',
    'pmt': 'PfS3',
    'kelch': 'PfS4',
    '6pgd': 'PfS5',
    'pure': 'NgS1',
    'barrel': 'NgS2',
    'is110': 'NgS3',
    'c39': 'NgS4',
}
results_df['segment_name'] = results_df.gene.map(gene_seg_map)
segment_summary = results_df.groupby(['species', 'segment_name']).agg(
    total_var = pd.NamedAgg(column='is_sig', aggfunc="count"),
    sig_count = pd.NamedAgg(column='is_sig', aggfunc="sum"),
).reset_index(drop=False)
segment_summary

,species,segment_name,total_var,sig_count
0,neisseria,NgS1,32,3
1,neisseria,NgS2,4,1
2,neisseria,NgS3,10,0
3,neisseria,NgS4,46,0
4,plasmodium,PfS1,12,1
5,plasmodium,PfS2,16,1
6,plasmodium,PfS3,12,5
7,plasmodium,PfS4,14,5
8,plasmodium,PfS5,22,4


In [18]:
segment_summary.to_csv(data_path.parent / 'segment_summary.csv', index=False)